<a href="https://colab.research.google.com/github/AnshuKamath/DB-Analytics-Assignment/blob/main/Section3_MongoDB_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "_id": ObjectId("5f8a7b234d9a6c2f3456789a"),
  "name": "John Smith",
  "gender": "Male",
  "age": 35,
  "phone_number": "+44123456789",
  "addresses": [
    {
      "_id": ObjectId("5f8a7b234d9a6c2f3456789b"),
      "house": "42",
      "street": "High Street",
      "city": "London",
      "post_code": "SW1A 1AA",
      "location": {
        "type": "Point",
        "coordinates": [-0.1275, 51.5072]
      },
      "is_default": true
    }
  ],
  "current_orders": [
    {
      "_id": ObjectId("5f8a7b234d9a6c2f3456789c"),
      "date": ISODate("2025-05-15T14:30:00Z"),
      "order_status": 2,  // 2 = Paid
      "order_details": {
        "total_cost": 54.99,
        "partner_id": ObjectId("5f8a7b234d9a6c2f3456789d"),
        "shipping_id": ObjectId("5f8a7b234d9a6c2f3456789b"),
        "supplier_id": ObjectId("5f8a7b234d9a6c2f3456789e")
      }
    }
  ],
  "recommended_products": [
    {
      "product_id": ObjectId("5f8a7b234d9a6c2f3456789f"),
      "avg_rating": 4.5
    }
  ],
  "created_at": ISODate("2024-01-15T10:20:30Z"),
  "updated_at": ISODate("2025-05-15T14:30:00Z")
}